In [1]:
from pony.orm import db_session, select
from be import db
from be.models import User, Course, Test, Question, Option, Tag, UserAnswer
import json

## Load seed data into db

In [2]:
db.bind(
    provider = 'mysql',
    host = 'localhost',
    user = 'its_admin',
    password = 'test123',
    db ='its-mysql'
)
db.generate_mapping(create_tables=True)

In [3]:
maths = Course(
    name='Maths'
)
geo = Course(
    name='Geography'
)
physics = Course(
    name='Physics'
)

def create_seed_data(file, course):
    with open(file, 'rb') as f:
        maths_questions = json.load(f)
        for q in maths_questions:
            tags = []
            for t in q['tags']:
                existing_tag = Tag.get(tag=t)
                if existing_tag:
                    tags.append(existing_tag)
                else:
                    tags.append(Tag(tag=t))
            question = Question(
                course=course, 
                question=q['question'],
                answer=q['answer'],
                difficulty=3,
                age_range="18-22",
                explanation='Sorry this is missing',
                tags=tags,
            )
            options = []
            for o in q['options']:
                options.append(Option(option=o,question=question))
            question.options = options
            
            
with db_session:
    create_seed_data('seed/maths.json', maths)
    create_seed_data('seed/geography.json', geo)
    create_seed_data('seed/physics.json', physics)

In [4]:
from datetime import date
with db_session:
    test = Test[1]
    userdob = test.for_user.dob
    user_age = date.today().year - user.dob.year
    course = Course[1]
    age_range = AgeRange[8, 11].in_range()
    print(age_range)
    questions = Question.select().filter(course=course).filter()
    available_questions = []
    for q in questions:
        if user not in q.tests.for_user:
            available_questions.append(q)
    print(available_questions)

ObjectNotFound: Test[1]

In [ ]:
with db_session:
    table = User.select()
    print(table.show())

In [ ]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List, Optional
import json
import re

class Question(BaseModel):
    question: str = Field(description="The question.")
    answer: str = Field(description="The answer to the question.")
    explanation: str = Field(description="The explanation of the answer.")
    difficulty: str = Field(description="The difficulty of the question on a scale of 1 to 5.")
    tags: List[str] = Field(description="The tags of the question describing keywords of the topic of question.")
    options: Optional[List[str]] = Field(description="The options of the question if it is a multiple choice question.")

class ListQuestions(BaseModel):
    questions: List[Question] = Field(description="The list of questions generated.")

parser = PydanticOutputParser(pydantic_object=ListQuestions)

some_json = """
{"questions":[
   {
      "question":"What is the square root of 25?",
      "answer":"5",
      "explanation":"The square root of 25 is 5 because 5 x 5 = 25.",
      "difficulty":"2",
      "tags":[
         "square root",
         "maths"
      ]
   },
   {
      "question":"What is the area of a circle with a radius of 5 cm?",
      "answer":"78.53981633974483",
      "explanation":"The area of a circle is calculated by multiplying pi (3.14) with the square of the radius (5 x 5). Therefore, the area of the circle is 3.14 x 25 = 78.53981633974483.",
      "difficulty":"3",
      "tags":[
         "area",
         "circle",
         "maths"
      ]
   },
   {
      "question":"What is the sum of the first 10 natural numbers?",
      "answer":"55",
      "explanation":"The sum of the first 10 natural numbers is 1 + 2 + 3 + 4 + 5 + 6 + 7 + 8 + 9 + 10 = 55.",
      "difficulty":"1",
      "tags":[
         "sum",
         "natural numbers",
         "maths"
      ]
   },
   {
      "question":"What is the value of x in the equation 3x + 5 = 22?",
      "answer":"7",
      "explanation":"To solve this equation, subtract 5 from both sides to get 3x = 17. Then divide both sides by 3 to get x = 17/3 = 7.",
      "difficulty":"2",
      "tags":[
         "equation",
         "maths"
      ]
   },
   {
      "question":"What is the value of x in the equation x^2 + 4x + 4 = 0?",
      "answer":"-2, 2",
      "explanation":"To solve this equation, use the quadratic formula: x = (-b +/- sqrt(b^2 - 4ac))/2a. In this case, a = 1, b = 4, and c = 4. Therefore, x = (-4 +/- sqrt(16 - 16))/2 = (-4 +/- 0)/2 = -2, 2.",
      "difficulty":"4",
      "tags":[
         "equation",
         "quadratic formula",
         "maths"
      ]
   },
   {
      "question":"What is the probability of rolling a 6 on a standard 6-sided die?",
      "answer":"1/6",
      "explanation":"The probability of rolling a 6 on a standard 6-sided die is 1/6 because there is only one 6 on the die and there are 6 possible outcomes.",
      "difficulty":"2",
      "tags":[
         "probability",
         "die",
         "maths"
      ]
   },
   {
      "question":"What is the slope of the line y = 3x + 2?",
      "answer":"3",
      "explanation":"The slope of the line y = 3x + 2 is 3 because the coefficient of x is 3.",
      "difficulty":"1",
      "tags":[
         "slope",
         "line",
         "maths"
      ]
   },
   {
      "question":"What is the value of x in the equation x^2 - 4x + 3 = 0?",
      "answer":"1, 3",
      "explanation":"To solve this equation, use the quadratic formula: x = (-b +/- sqrt(b^2 - 4ac))/2a. In this case, a = 1, b = -4, and c = 3. Therefore, x = (-(-4) +/- sqrt((-4)^2 - 4(1)(3)))/2 = (4 +/- sqrt(16 - 12))/2 = 1, 3.",
      "difficulty":"4",
      "tags":[
         "equation",
         "quadratic formula",
         "maths"
      ]
   },
   {
      "question":"What is the value of x in the equation 2x + 3 = 11?",
      "answer":"4",
      "explanation":"To solve this equation, subtract 3 from both sides to get 2x = 8. Then divide both sides by 2 to get x = 8/2 = 4.",
      "difficulty":"2",
      "tags":[
         "equation",
         "maths"
      ]
   },
   {
      "question":"What is the sum of the interior angles of a triangle?",
      "answer":"180 degrees",
      "explanation":"The sum of the interior angles of a triangle is 180 degrees because the sum of the angles of any triangle is 180 degrees.",
      "difficulty":"1",
      "tags":[
         "sum",
         "interior angles",
         "triangle",
         "maths"
      ]
   },
   {
      "question":"What is the probability of rolling a 4 on a standard 6-sided die?",
      "answer":"1/6",
      "explanation":"The probability of rolling a 4 on a standard 6-sided die is 1/6 because there is only one 4 on the die and there are 6 possible outcomes.",
      "difficulty":"2",
      "tags":[
         "probability",
         "die",
         "maths"
      ]
   },
   {
      "question":"What is the area of a triangle with a base of 10 cm and a height of 5 cm?",
      "answer":"25 cm^2",
      "explanation":"The area of a triangle is calculated by multiplying the base by the height and then dividing by 2. Therefore, the area of the triangle is (10 x 5)/2 = 25 cm^2.",
      "difficulty":"3",
      "tags":[
         "area",
         "triangle",
         "maths"
      ]
   },
   {
      "question":"What is the value of x in the equation x^2 + 6x + 9 = 0?",
      "answer":"-3, -3",
      "explanation":"To solve this equation, use the quadratic formula: x = (-b +/- sqrt(b^2 - 4ac))/2a. In this case, a = 1, b = 6, and c = 9. Therefore, x = (-6 +/- sqrt(36 - 36))/2 = (-6 +/- 0)/2 = -3, -3.",
      "difficulty":"4",
      "tags":[
         "equation",
         "quadratic formula",
         "maths"
      ]
   },
   {
      "question":"What is the circumference of a circle with a radius of 5 cm?",
      "answer":"31.41592653589793 cm",
      "explanation":"The circumference of a circle is calculated by multiplying pi (3.14) with the diameter (2 x 5). Therefore, the circumference of the circle is 3.14 x 10 = 31.41592653589793 cm.",
      "difficulty":"3",
      "tags":[
         "circumference",
         "circle",
         "maths"
      ]
   },
   {
      "question":"What is the probability of rolling an even number on a standard 6-sided die?",
      "answer":"1/2",
      "explanation":"The probability of rolling an even number on a standard 6-sided die is 1/2 because there are three even numbers (2, 4, 6) on the die and there are 6 possible outcomes.",
      "difficulty":"2",
      "tags":[
         "probability",
         "die",
         "maths"
      ]
   },
   {
      "question":"What is the value of x in the equation x/2 + 3 = 7?",
      "answer":"10",
      "explanation":"To solve this equation, subtract 3 from both sides to get x/2 = 4. Then multiply both sides by 2 to get x = 4 x 2 = 10.",
      "difficulty":"2",
      "tags":[
         "equation",
         "maths"
      ]
   },
   {
      "question":"What is the probability of rolling a number greater than 4 on a standard 6-sided die?",
      "answer":"2/3",
      "explanation":"The probability of rolling a number greater than 4 on a standard 6-sided die is 2/3 because there are two numbers greater than 4 (5 and 6) on the die and there are 6 possible outcomes.",
      "difficulty":"2",
      "tags":[
         "probability",
         "die",
         "maths"
      ]
   },
   {
      "question":"What is the value of x in the equation x^2 - 6x + 9 = 0?",
      "answer":"3, 3",
      "explanation":"To solve this equation, use the quadratic formula: x = (-b +/- sqrt(b^2 - 4ac))/2a. In this case, a = 1, b = -6, and c = 9. Therefore, x = (-(-6) +/- sqrt((-6)^2 - 4(1)(9)))/2 = (6 +/- sqrt(36 - 36))/2 = 3, 3.",
      "difficulty":"4",
      "tags":[
         "equation",
         "quadratic formula",
         "maths"
      ]
   },
   {
      "question":"What is the probability of rolling an odd number on a standard 6-sided die?",
      "answer":"1/2",
      "explanation":"The probability of rolling an odd number on a standard 6-sided die is 1/2 because there are three odd numbers (1, 3, 5) on the die and there are 6 possible outcomes.",
      "difficulty":"2",
      "tags":[
         "probability",
         "die",
         "maths"
      ]
   }
]
}"""

lq = parser.parse(some_json)

for q in lq.questions:
    print(q)

In [8]:
with db_session:
    print(Test.select().show())

id|course|for_user|ready|completed
--+------+--------+-----+---------
None


In [9]:
with db_session:
    print(User.select().show())


id|username|password                                                            
--+--------+--------------------------------------------------------------------
1 |ross    |pbkdf2:sha256:600000$Ca8zFeQr4XQnlLxQ$2d0025b2dec190a3ffb9ed49643...
None
